<a href="https://colab.research.google.com/github/andrewgcodes/FalconStreaming/blob/main/Falcon40B_Instruct_Streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Falcon 40B Instruct with Streaming

You will need Google Colab Pro+ in order to manage such a large model.

If you don't have it, I recommend trying the 7B model instead, which will probably work on regular Colab or Colab Pro.

Streaming by default prints out all words onto one line. You may want to copy and paste the output into a doc to see it better.

## Imports
You can ignore this. Just imports.

In [1]:
!pip install git+https://www.github.com/huggingface/transformers
!pip install git+https://github.com/huggingface/accelerate

!pip install bitsandbytes

!pip install einops

from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import torch


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://www.github.com/huggingface/transformers to /tmp/pip-req-build-tvip4_br
  Running command git clone --filter=blob:none --quiet https://www.github.com/huggingface/transformers /tmp/pip-req-build-tvip4_br
  Resolved https://www.github.com/huggingface/transformers to commit ff4c0fc7d2e8df5fb5971f9d8dbd1108edabf65b
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.0 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.30.0.dev0-py3-none-any.whl size=7165487 sha256=29757d708ecb5263afe592c5a0cd77994b0202fd21d6514c4cac778f82c13ebc
  Stored in directory: /tmp

## Model loading
You can swap the model_path to tiiuae/falcon-7b-instruct

In [2]:
#tiiuae/falcon-7b-instruct works too! Smaller and faster. But much worse quality.
model_path="tiiuae/falcon-40b-instruct"
 
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, load_in_4bit=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-40b-instruct")

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-40b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-40b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

## Set your prompt.

In [19]:
input_text = "Tell a story about a magical fish in the UAE:"

# Basic Streaming (uses most likely next token)

In [20]:
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

next_input = input_ids
max_length = 80  # Change this to your desired output length. Too long could cause an OOM Out of Memory error!
current_length = input_ids.shape[1]

while True:
    if current_length >= max_length:  # Check if we've reached the length limit
        break

    output = model(next_input)
    next_token_logits = output.logits[:, -1, :]
    next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(0)
    print(tokenizer.decode(next_token_id[0].cpu().tolist(), skip_special_tokens=True), end='', flush=True)

    next_input = torch.cat([next_input, next_token_id.to("cuda")], dim=-1)

    current_length += 1

    if next_token_id[0].item() == tokenizer.eos_token_id:
        break



Once upon a time, there was a magical fish that lived in the waters of the UAE. This fish had the power to grant wishes to anyone who caught it. One day, a fisherman caught the fish and asked for a wish. The fish granted his wish and he became rich beyond his wildest dreams. The fisherman was so happy that

# Top-k

In [18]:
import torch
import torch.nn.functional as F

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

next_input = input_ids
max_length = 100  # Change this to your desired output length
current_length = input_ids.shape[1]
k = 10  # Number of tokens to sample from. Adjust as necessary. Greater k = greater variability.

while True:
    if current_length >= max_length:  # Check if we've reached the length limit
        break

    output = model(next_input)
    next_token_logits = output.logits[:, -1, :]
    
    # Apply top-k sampling
    indices_to_remove = next_token_logits < torch.topk(next_token_logits, k)[0][..., -1, None]
    next_token_logits[indices_to_remove] = float('-inf')
    next_token_id = torch.multinomial(F.softmax(next_token_logits, dim=-1), num_samples=1)

    print(tokenizer.decode(next_token_id[0].cpu().tolist(), skip_special_tokens=True), end='', flush=True)

    next_input = torch.cat([next_input, next_token_id.to("cuda")], dim=-1)

    current_length += 1

    if next_token_id[0].item() == tokenizer.eos_token_id:
        break



The glowing fish was a rare creature that could only be found deep in the ocean's dark depths. As it swam, it emitted a soft blue light that illuminated its surroundings, making it easy to navigate in the darkness. Its scales shimmered like diamonds, and its eyes were a bright green that seemed to glow brighter with every passing moment. People would tell stories of encountering the glowing fish on their travels, and it became a mythical creature with its

## Beam Search (with streaming)

In [24]:
import torch

def beam_search(input_ids, model, tokenizer, beam_size, max_length):
    beam = [(input_ids, 0)]  # Each element in the beam is a tuple (input_ids, score)

    for step in range(max_length):
        new_beam = []
        print(f"\n=== Step {step + 1} ===")

        for beam_id, (prev_input_ids, prev_score) in enumerate(beam):
            output = model(prev_input_ids)
            next_token_logits = output.logits[:, -1, :]
            next_token_probs = torch.softmax(next_token_logits, dim=-1)

            top_probs, top_indices = torch.topk(next_token_probs, beam_size)
            
            for i in range(beam_size):
                next_input_ids = torch.cat([prev_input_ids, top_indices[0][i].unsqueeze(0).unsqueeze(0)], dim=-1)
                score = prev_score + torch.log(top_probs[0][i]).item()
                
                new_beam.append((next_input_ids, score))

        beam = sorted(new_beam, key=lambda x: x[1], reverse=True)[:beam_size]

        # Print out the current state of each beam
        for beam_id, b in enumerate(beam):
            print(f"\nBeam {beam_id + 1} Score: {b[1]}")
            print(tokenizer.decode(b[0][0].cpu().tolist(), skip_special_tokens=True))

    return beam

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

beam_size = 5  # Adjust as necessary
max_length = 100  # Adjust as necessary

beam = beam_search(input_ids, model, tokenizer, beam_size, max_length)

# Print out the highest-scoring output
print("\n=== Final Output ===")
print(f"\nBest Beam Score: {beam[0][1]}")
print(tokenizer.decode(beam[0][0][0].cpu().tolist(), skip_special_tokens=True))



=== Step 1 ===

Beam 1 Score: -0.0187225341796875
Tell a story about a magical fish in the UAE:


Beam 2 Score: -5.06640625
Tell a story about a magical fish in the UAE: The

Beam 3 Score: -6.37109375
Tell a story about a magical fish in the UAE: 

Beam 4 Score: -6.66796875
Tell a story about a magical fish in the UAE: the

Beam 5 Score: -7.16796875
Tell a story about a magical fish in the UAE: A

=== Step 2 ===

Beam 1 Score: -0.9928436279296875
Tell a story about a magical fish in the UAE:
Once

Beam 2 Score: -1.5079803466796875
Tell a story about a magical fish in the UAE:
In

Beam 3 Score: -2.2277069091796875
Tell a story about a magical fish in the UAE:
As

Beam 4 Score: -2.3839569091796875
Tell a story about a magical fish in the UAE:
The

Beam 5 Score: -2.5714569091796875
Tell a story about a magical fish in the UAE:
There

=== Step 3 ===

Beam 1 Score: -1.0165557861328125
Tell a story about a magical fish in the UAE:
Once upon

Beam 2 Score: -1.5840301513671875
Tell a story ab